# Etapa 4

## Dependências

In [32]:
import pandas as pd
import numpy as np

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, precision_recall_curve

## Preparação dos Dados

In [33]:
# Carregamento dos dados processados
df = pd.read_parquet("NYPD-Tidy.parquet")
df.shape

# Remove as colunas latitude e longitude
X = df.drop(columns=["STATISTICAL_MURDER_FLAG", "Latitude", "Longitude"], errors="ignore")
y = df["STATISTICAL_MURDER_FLAG"]


# Identifica os tipos de colunas
categoricas = X.select_dtypes(include=["object", "category"]).columns.tolist()
numericas = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Pipeline com imputação + transformação
preprocessador = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), numericas),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), categoricas)
])


In [34]:
# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


## Modelos Testados

### Regressão Logística, Random Forest e XGBoost (Sem Ajustes)

In [35]:
# Função para treinar, avaliar e imprimir resultados de um modelo
def avaliar(modelo, nome):
    # Cria um pipeline com duas etapas:
    # 1. 'prep': o pré-processador (ex: OneHotEncoder, StandardScaler etc.)
    # 2. 'model': o modelo de aprendizado de máquina passado como argumento
    pipe = Pipeline(steps=[("prep", preprocessador), ("model", modelo)])

    # Treina o pipeline nos dados de treino
    pipe.fit(X_treino, y_treino)

    # Faz previsões nos dados de teste (rótulos binários)
    pred = pipe.predict(X_teste)

    # Calcula as probabilidades de classe positiva (para métricas baseadas em score)
    prob = pipe.predict_proba(X_teste)[:, 1]

    # Exibe nome do modelo e relatório de classificação (precision, recall, f1-score, support)
    print(f"\n===== {nome} =====")
    print(classification_report(y_teste, pred))

    # Exibe a métrica ROC-AUC com base nas probabilidades preditas
    print("ROC-AUC:", roc_auc_score(y_teste, prob))

    # Retorna o pipeline treinado, caso se deseje reutilizar
    return pipe

# Avaliação com Regressão Logística (com máximo de 1000 iterações para garantir convergência)
modelo_lr = avaliar(LogisticRegression(max_iter=1000), "Regressão Logística")

# Avaliação com Random Forest (usando semente para reprodutibilidade)
modelo_rf = avaliar(RandomForestClassifier(random_state=42), "Random Forest")

# Avaliação com XGBoost (definindo a métrica de avaliação como logloss)
modelo_xgb = avaliar(XGBClassifier(eval_metric='logloss'), "XGBoost")



===== Regressão Logística =====
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      4595
           1       0.45      0.09      0.15      1104

    accuracy                           0.80      5699
   macro avg       0.64      0.53      0.52      5699
weighted avg       0.75      0.80      0.75      5699

ROC-AUC: 0.6803150281496901

===== Random Forest =====
              precision    recall  f1-score   support

           0       0.80      0.94      0.87      4595
           1       0.15      0.04      0.07      1104

    accuracy                           0.77      5699
   macro avg       0.48      0.49      0.47      5699
weighted avg       0.68      0.77      0.71      5699

ROC-AUC: 0.6368048327577234

===== XGBoost =====
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      4595
           1       0.48      0.06      0.10      1104

    accuracy                           0.81  

### Após Ajustes

#### Regressão Logística Final

In [36]:
# Pipeline com UNDERSAMPLING
pipe_lr = Pipeline(steps=[
    ("prep", preprocessador),                                  # Etapa de pré-processamento (ex: encoding, scaling etc.)
    ("undersample", RandomUnderSampler(random_state=42)),      # Redução da classe majoritária (undersampling)
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))  # Modelo: regressão logística
    # 'class_weight="balanced"' reforça o tratamento de desbalanceamento mesmo após o undersampling
])

# Grade de hiperparâmetros para busca por validação cruzada
param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10],          # C: força da regularização (quanto menor, mais regularização)
    "model__penalty": ["l2"],               # Penalidade: L2 (ridge)
    "model__solver": ["lbfgs"],             # Solver usado para otimização (funciona bem com L2)
}

# Busca dos melhores hiperparâmetros com validação cruzada (3 folds)
grid_lr = GridSearchCV(
    pipe_lr, param_grid_lr,
    scoring="f1",              # Otimiza F1-score (balanceia precisão e recall)
    cv=3,                      # 3 folds de validação cruzada
    verbose=1,                 # Mostra o andamento do processo
    n_jobs=-1                  # Usa todos os núcleos da CPU para acelerar
)
grid_lr.fit(X_treino, y_treino)  # Treina a busca com undersampling

# Resultados do tuning
print("Melhores parâmetros (LogReg + UnderSampling):", grid_lr.best_params_)
print("Melhor F1-score (CV):", grid_lr.best_score_)

# Avaliação no conjunto de teste
best_lr = grid_lr.best_estimator_                 # Melhor pipeline encontrado
pred = best_lr.predict(X_teste)                   # Previsões de classe
prob = best_lr.predict_proba(X_teste)[:, 1]       # Probabilidades da classe positiva

# Impressão dos resultados finais no teste
print("\n== Regressão Logística + UnderSampling ==")
print(classification_report(y_teste, pred))       # Relatório com precisão, recall, F1 e suporte
print("ROC-AUC:", roc_auc_score(y_teste, prob))   # Curva ROC-AUC (discriminação global)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Melhores parâmetros (LogReg + UnderSampling): {'model__C': 1, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
Melhor F1-score (CV): 0.3816119637143389

== Regressão Logística + UnderSampling ==
              precision    recall  f1-score   support

           0       0.87      0.62      0.72      4595
           1       0.28      0.62      0.39      1104

    accuracy                           0.62      5699
   macro avg       0.58      0.62      0.55      5699
weighted avg       0.76      0.62      0.66      5699

ROC-AUC: 0.6672745067890429


#### Random Forest Final

In [37]:
# Pipeline com Random Forest otimizado para eficiência
pipe_rf_fast = Pipeline([
    ("prep", preprocessador),  # Etapa de pré-processamento (ex: encoding, scaling etc.)
    ("model", RandomForestClassifier(
        n_estimators=200,            # Número de árvores (pode ser 100 para ganhar velocidade)
        max_depth=12,                # Profundidade máxima das árvores (controla overfitting)
        min_samples_leaf=2,          # Tamanho mínimo da folha (evita árvores muito ramificadas)
        min_samples_split=5,         # Mínimo de amostras para dividir um nó interno
        max_features="sqrt",         # Nº de features considerado por split (raiz quadrada)
        bootstrap=True,              # Usa amostragem com reposição (bagging)
        max_samples=0.6,             # Subamostra de dados por árvore (60% das amostras)
        class_weight="balanced_subsample",  # Corrige desbalanceamento em cada árvore
        n_jobs=-1,                   # Usa todos os núcleos disponíveis
        random_state=42             # Reprodutibilidade
    ))
])

# Espaço de busca para tuning dos hiperparâmetros (mais leve)
param_grid_fast = {
    "model__n_estimators": [100, 200],             # Nº de árvores
    "model__max_depth": [8, 12],                   # Profundidade das árvores
    "model__max_samples": [0.5, 0.6, 0.7],         # Proporção de amostras por árvore
    "model__min_samples_leaf": [2, 4],             # Tamanho mínimo da folha
    "model__max_features": ["sqrt", 0.3],          # Nº de features por split
}

# Validação cruzada estratificada (mantém proporções das classes)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Busca dos melhores hiperparâmetros com PR-AUC como métrica (ideal p/ desbalanceamento)
grid_fast = GridSearchCV(
    pipe_rf_fast,
    param_grid_fast,
    scoring="average_precision",   # Otimiza área sob a curva de precisão-recall
    cv=cv,
    n_jobs=-1,
    verbose=0
)

# Treina o modelo com busca de hiperparâmetros
grid_fast.fit(X_treino, y_treino)

# Melhor pipeline encontrado
best = grid_fast.best_estimator_

# Avaliação no conjunto de teste
prob = best.predict_proba(X_teste)[:,1]       # Probabilidade da classe positiva
pred = (prob >= 0.5).astype(int)              # Threshold padrão (0.5)

# Impressão dos resultados
print("Melhores params (RF-Lite):", grid_fast.best_params_)  # Hiperparâmetros otimizados
print("PR-AUC:", average_precision_score(y_teste, prob))     # Área sob a curva precisão-recall
print("ROC-AUC:", roc_auc_score(y_teste, prob))              # Área sob a curva ROC
print(classification_report(y_teste, pred))                  # Métricas: precisão, recall e F1


Melhores params (RF-Lite): {'model__max_depth': 8, 'model__max_features': 0.3, 'model__max_samples': 0.7, 'model__min_samples_leaf': 4, 'model__n_estimators': 200}
PR-AUC: 0.3034867155166983
ROC-AUC: 0.6558064649666462
              precision    recall  f1-score   support

           0       0.87      0.61      0.72      4595
           1       0.28      0.63      0.38      1104

    accuracy                           0.61      5699
   macro avg       0.57      0.62      0.55      5699
weighted avg       0.76      0.61      0.65      5699



#### XGBoost Final

In [38]:
# Cálculo do peso da classe positiva (para lidar com desbalanceamento)
neg = int((y_treino == 0).sum())  # Total da classe negativa
pos = int((y_treino == 1).sum())  # Total da classe positiva
spw = max(1.0, neg / max(1, pos))  # Peso para XGBoost (evita divisão por zero)

# Pipeline com XGBoost e pré-processamento
pipe_xgb = Pipeline(steps=[
    ("prep", preprocessador),  # Aplicação do ColumnTransformer
    ("model", XGBClassifier(
        objective="binary:logistic",  # Tarefa binária com saída probabilística
        eval_metric="aucpr",          # Foco em PR-AUC (melhor para dados desbalanceados)
        tree_method="hist",           # Algoritmo mais rápido para grandes conjuntos
        n_jobs=-1,                    # Usa todos os núcleos disponíveis
        scale_pos_weight=spw,        # Aplica ponderação automática para balancear classes
        random_state=42
    ))
])

# Espaço de busca para tuning dos hiperparâmetros
param_dist = {
    "model__n_estimators":     [300, 600],
    "model__learning_rate":    [0.03, 0.1],
    "model__max_depth":        [4, 8],
    "model__min_child_weight": [1, 3],
    "model__subsample":        [0.6, 1.0],
    "model__colsample_bytree": [0.6, 1.0],
    "model__reg_lambda":       [1.0, 10.0],
    "model__reg_alpha":        [0.0, 1.0],
    "model__gamma":            [0.0, 1.0],
}

# Tuning com validação cruzada (3-fold)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
rnd = RandomizedSearchCV(
    pipe_xgb,
    param_distributions=param_dist,
    n_iter=4,                        # Limita para 4 combinações (mais rápido)
    scoring="f1",                   # Otimiza F1-score da classe positiva
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)
rnd.fit(X_treino, y_treino)

best_xgb = rnd.best_estimator_      # Melhor pipeline treinado
melhores_param = rnd.best_params_   # Hiperparâmetros escolhidos
melhor_score = rnd.best_score_      # Melhor F1 médio obtido na CV

# Função auxiliar para encontrar o melhor threshold com base em F1 via CV
def escolher_threshold_cv(estimator, X_train, y_train, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    ths = []
    for tr, va in skf.split(X_train, y_train):
        estimator.fit(X_train.iloc[tr], y_train.iloc[tr])
        p = estimator.predict_proba(X_train.iloc[va])[:, 1]
        prec, rec, thr = precision_recall_curve(y_train.iloc[va], p)
        f1 = 2*prec*rec/(prec+rec+1e-12)
        i = int(np.nanargmax(f1))  # Índice do melhor F1
        th = thr[i-1] if i > 0 and i-1 < len(thr) else 0.5
        ths.append(float(th))
    return float(np.median(ths))  # Threshold final = mediana entre folds

thr_star = escolher_threshold_cv(best_xgb, X_treino, y_treino, n_splits=5)

# Avaliação final no conjunto de teste
prob = best_xgb.predict_proba(X_teste)[:, 1]   # Probabilidade da classe positiva
predCV = (prob >= thr_star).astype(int)        # Aplicação do threshold otimizado

# Impressão dos resultados
print(f"\nMelhores parâmetros (XGBoost + Tuning): {melhores_param}")
print(f"Melhor F1-score (CV): {melhor_score}")

print("\n== XGBoost Tunado ==")
print(classification_report(y_teste, predCV))                      # Métricas finais no teste
print("ROC-AUC:", roc_auc_score(y_teste, prob))                   # Área sob a curva ROC
print("PR-AUC :", average_precision_score(y_teste, prob))         # Área sob a curva PR


Fitting 3 folds for each of 4 candidates, totalling 12 fits

Melhores parâmetros (XGBoost + Tuning): {'model__subsample': 0.6, 'model__reg_lambda': 1.0, 'model__reg_alpha': 1.0, 'model__n_estimators': 600, 'model__min_child_weight': 3, 'model__max_depth': 4, 'model__learning_rate': 0.1, 'model__gamma': 0.0, 'model__colsample_bytree': 1.0}
Melhor F1-score (CV): 0.39674845055961

== XGBoost Tunado ==
              precision    recall  f1-score   support

           0       0.87      0.64      0.74      4595
           1       0.28      0.59      0.38      1104

    accuracy                           0.63      5699
   macro avg       0.58      0.62      0.56      5699
weighted avg       0.75      0.63      0.67      5699

ROC-AUC: 0.6718528725300027
PR-AUC : 0.32716825702824337


## Comparação Quantitativa

In [46]:
# Resultados antes e depois de cada modelo
dados = {
    "Modelo": [
        "LogReg (baseline)",
        "LogReg + UnderSampling + Tuning",
        "Random Forest (baseline)",
        "Random Forest + Tuning",
        "XGBoost (baseline)",
        "XGBoost + Tuning + Threshold CV"
    ],
    "Precision (classe 1)": [0.45, 0.28, 0.15, 0.28, 0.48, 0.28],
    "Recall (classe 1)":    [0.09, 0.62, 0.04, 0.63, 0.06, 0.59],
    "F1-score (classe 1)":  [0.15, 0.39, 0.07, 0.38, 0.10, 0.38],
    "ROC-AUC":              [0.68, 0.67, 0.64, 0.66, 0.68, 0.67],
    "PR-AUC":               [None, None, None, 0.30, None, 0.33]
}

df_comp = pd.DataFrame(dados)
df_comp.set_index("Modelo", inplace=True)
df_comp


,Precision (classe 1),Recall (classe 1),F1-score (classe 1),ROC-AUC,PR-AUC
Modelo,,,,,
LogReg (baseline),0.45,0.09,0.15,0.68,NaN
LogReg + UnderSampling + Tuning,0.28,0.62,0.39,0.67,NaN
Random Forest (baseline),0.15,0.04,0.07,0.64,NaN
Random Forest + Tuning,0.28,0.63,0.38,0.66,0.30
XGBoost (baseline),0.48,0.06,0.10,0.68,NaN
XGBoost + Tuning + Threshold CV,0.28,0.59,0.38,0.67,0.33


## Variáveis Mais Influentes

In [63]:
# Função auxiliar para obter os nomes das features após o preprocessador
def get_feature_names(prep):
    return prep.get_feature_names_out()

# ==============================
# 1. Regressão Logística
# ==============================
modelo_lr = best_lr.named_steps["model"]
prep_lr = best_lr.named_steps["prep"]
features_lr = get_feature_names(prep_lr)

coef = modelo_lr.coef_.flatten()
assert len(features_lr) == len(coef), "Número de coeficientes não bate com número de features"

df_coef = pd.DataFrame({
    "feature": features_lr,
    "coef": coef,
    "abs_coef": np.abs(coef)
}).sort_values(by="abs_coef", ascending=False)

print("Top 15 variáveis mais impactantes - Regressão Logística:")
print(df_coef.head(15))


# ==============================
# 2. Random Forest
# ==============================
best_rf = grid_fast.best_estimator_
modelo_rf = best_rf.named_steps["model"]
prep_rf = best_rf.named_steps["prep"]
features_rf = get_feature_names(prep_rf)

importancias_rf = modelo_rf.feature_importances_
assert len(features_rf) == len(importancias_rf), "Número de importâncias ≠ número de features"

df_rf = pd.DataFrame({
    "feature": features_rf,
    "importance": importancias_rf
}).sort_values(by="importance", ascending=False)

print("\nTop 15 variáveis mais importantes - Random Forest:")
print(df_rf.head(15))


# ==============================
# 3. XGBoost
# ==============================
modelo_xgb = best_xgb.named_steps["model"]
prep_xgb = best_xgb.named_steps["prep"]
features_xgb = get_feature_names(prep_xgb)

importancias_xgb = modelo_xgb.feature_importances_
assert len(features_xgb) == len(importancias_xgb), "Número de importâncias ≠ número de features"

df_xgb = pd.DataFrame({
    "feature": features_xgb,
    "importance": importancias_xgb
}).sort_values(by="importance", ascending=False)

print("\nTop 15 variáveis mais importantes - XGBoost:")
print(df_xgb.head(15))


Top 15 variáveis mais impactantes - Regressão Logística:
                                                 feature      coef  abs_coef
1476                         cat__PERP_AGE_GROUP_UNKNOWN -2.590643  2.590643
1480                               cat__PERP_SEX_UNKNOWN  2.276037  2.276037
956                             cat__OCCUR_TIME_16:25:00 -1.594320  1.594320
6329   cat__Lon_Lat_POINT (-73.90317908399999 40.8248...  1.583077  1.583077
10450  cat__Lon_Lat_POINT (-73.94364075199996 40.6892...  1.492544  1.492544
24                              cat__OCCUR_TIME_00:19:00 -1.398520  1.398520
1243                            cat__OCCUR_TIME_21:12:00  1.382360  1.382360
986                             cat__OCCUR_TIME_16:55:00 -1.373565  1.373565
1115                            cat__OCCUR_TIME_19:04:00 -1.347919  1.347919
11154  cat__Lon_Lat_POINT (-73.95082167399994 40.6488...  1.337668  1.337668
10020  cat__Lon_Lat_POINT (-73.93999542099994 40.7887...  1.285601  1.285601
8829           cat_

## Conclusão

Durante o desenvolvimento do projeto, observou-se que abordagens padrão e desbalanceadas, como Regressão Logística, Random Forest e XGBoost sem ajuste de classes, apresentaram desempenho insatisfatório, especialmente na detecção da classe minoritária, responsável por representar os tiroteios letais. Esses modelos obtiveram baixos valores de F1-score e recall para a classe 1, evidenciando a necessidade de estratégias específicas para lidar com o desbalanceamento de classes. Nesse sentido, técnicas como o uso de class_weight="balanced" e, sobretudo, o undersampling via RandomUnderSampler se mostraram eficazes, promovendo melhorias expressivas em recall e F1-score da classe de interesse, mesmo com alguma perda de acurácia geral. Tais resultados indicam que, embora o desempenho para a classe majoritária seja impactado, os modelos tornam-se mais justos e sensíveis à detecção de eventos relevantes para a análise.

Com o avanço das iterações, a aplicação de Random Forest com tuning foi explorada como alternativa eficiente do ponto de vista computacional, alcançando F1-score competitivo e boa capacidade de generalização. No entanto, persistiu o problema de baixa precisão para a classe minoritária, indicando a presença de elevado número de falsos positivos. Em seguida, a utilização do XGBoost com tuning e definição de threshold personalizado por validação cruzada revelou-se a abordagem mais equilibrada, atingindo o melhor F1-score no processo de validação e resultados consistentes no conjunto de teste. A escolha de um limiar de decisão otimizado, em vez de utilizar o valor padrão de 0.5, foi crucial para adaptar o modelo à distribuição real do problema e maximizar a efetividade da classificação.

Ao longo do processo, foram empregadas métricas de performance apropriadas ao problema de classificação binária desbalanceada, como F1-score, recall, ROC-AUC e PR-AUC, de forma a capturar a habilidade dos modelos em identificar corretamente os tiroteios letais. A regressão logística, usada como baseline, apresentou desempenho limitado (F1 = 0.15; recall = 0.09), sendo posteriormente aprimorada com técnicas de undersampling e tuning, o que elevou seu F1-score para 0.39. Modelos baseados em árvores, como Random Forest e XGBoost, atingiram F1-scores similares (até 0.38), com XGBoost se destacando em PR-AUC (0.33), evidenciando maior robustez para o contexto do problema.

No que tange à explicabilidade, a regressão logística permitiu análise direta dos coeficientes, revelando que variáveis como horário do crime e categorias desconhecidas (“UNKNOWN”/"Null") relativas ao sexo e idade do agressor estavam entre as mais influentes. Em contrapartida, Random Forest e XGBoost apontaram como mais importantes os atributos relacionados à localização geográfica, ao perfil do autor e ao tipo de ocorrência, contribuindo para a validação das hipóteses de que o contexto espacial e temporal, bem como o perfil das partes envolvidas, influenciam significativamente a letalidade dos tiroteios. Ainda assim, notou-se um claro trade-off entre interpretabilidade e complexidade: modelos lineares oferecem maior transparência, enquanto modelos de árvore, embora mais eficazes, exigem técnicas adicionais para interpretação. Por fim, limitações como o desbalanceamento residual, a presença de dados incompletos e a potencial limitação na generalização dos modelos para novos contextos foram reconhecidas, reforçando a necessidade de um equilíbrio criterioso entre desempenho preditivo e capacidade explicativa na escolha do modelo final.